# Documentation - Links from ODS

### API Overview
https://digital.nhs.uk/services/organisation-data-service/apis-for-the-organisation-data-service#section-4

### Search endpoint
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-endpoint

Search parameters
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-parameters#name-substring-search-

### Ogranisation endpoint 
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/organisation-endpoint

Org succession explanation
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/how-to-understand-organisation-succession

More endpoints
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/ods-ord-api-endpoints

Implementation guide and links
https://directory.spineservices.nhs.uk/ODSAPISuite

In [1]:
import pandas as pd
import requests
import json

In [2]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [3]:
#example search endpoint
response = requests.get("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PostCode=LS14BT")

In [4]:
#example organisation endpoint
ODScode = "505"
response = requests.get("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/"+ODScode+"?")

In [5]:
#preview API response
jprint(response.json())

{
    "Organisation": {
        "Date": [
            {
                "End": "1998-03-31",
                "Start": "1980-01-01",
                "Type": "Operational"
            }
        ],
        "GeoLoc": {
            "Location": {
                "AddrLn1": "**UNKNOWN**",
                "Country": "ENGLAND"
            }
        },
        "LastChangeDate": "2013-05-08",
        "Name": "NOTTINGHAMSHIRE",
        "OrgId": {
            "assigningAuthorityName": "HSCIC",
            "extension": "505",
            "root": "2.16.840.1.113883.2.1.3.2.4.18.48"
        },
        "Roles": {
            "Role": [
                {
                    "Date": [
                        {
                            "End": "1998-03-31",
                            "Start": "1980-01-01",
                            "Type": "Operational"
                        }
                    ],
                    "Status": "Inactive",
                    "id": "RO141",
                    "pri

In [6]:
data = response.json()

In [7]:
#format whole json reply as normalised dataframe
df = pd.DataFrame.from_dict(pd.json_normalize(response.json()), orient='columns')
df

,Organisation.Name,Organisation.Date,Organisation.OrgId.root,Organisation.OrgId.assigningAuthorityName,Organisation.OrgId.extension,Organisation.Status,Organisation.LastChangeDate,Organisation.orgRecordClass,Organisation.refOnly,Organisation.GeoLoc.Location.AddrLn1,Organisation.GeoLoc.Location.Country,Organisation.Roles.Role
0,NOTTINGHAMSHIRE,"[{'Type': 'Operational', 'Start': '1980-01-01'...",2.16.840.1.113883.2.1.3.2.4.18.48,HSCIC,505,Inactive,2013-05-08,RC1,True,**UNKNOWN**,ENGLAND,"[{'id': 'RO141', 'uniqueRoleId': 57138, 'prima..."


In [9]:
#create empty dataframe
df_succ = pd.DataFrame(columns = ["SiteODScode", "SuccessorODScode", "PredecessorODScode", "Date"])

#populate dataframe row by row
try:
    for suc in data["Organisation"]["Succs"]["Succ"]:

        site = str(data["Organisation"]["OrgId"]["extension"])
        typesp= (suc["Type"])
        if typesp == "Predecessor":
            successor = "-"
            predecessor = (suc["Target"]["OrgId"]["extension"])
        elif typesp == "Successor":
            successor = (suc["Target"]["OrgId"]["extension"])
            predecessor = "-"
        else:
            successor = "-"
            predecessor = "-"

        date = (suc["Date"][0]["Start"])

        df_succ = df_succ.append({'SiteODScode' : site , 
                                  'SuccessorODScode' : successor, 
                                  'PredecessorODScode' :predecessor, 
                                  'Date' : date}, ignore_index=True)
except:
        print( "No succession data available")
#view dataframe
df_succ

No succession data available


,SiteODScode,SuccessorODScode,PredecessorODScode,Date
